# 4. 신경망 학습

In [13]:
import numpy as np
import sys,os
from datasets.mnist import load_mnist
sys.path.append(os.pardir)

### 오차제곱합(Sum of Squares for error,SSE)

In [7]:
def sum_squares_error(y,t):
    return 0.5 * np.sum((y-t)**2)

In [8]:
t = [0,0,1,0,0,0,0,0,0,0] # 2 원-핫 인코딩

y = [0.1,0.05,0.6,.0,.05,.1,.0,.1,.0,.0]

sum_squares_error(np.array(y),np.array(t))


0.09750000000000003

In [9]:
y = [.1,.05,.1,.0,.05,.1,.0,.6,.0,.0] # 신경망의 출력값이 7이 가장 높은 경우 
sum_squares_error(np.array(y),np.array(t))


0.5975

### 교차 엔트로피 오차(Cross Entropy Error,CEE)

In [10]:
def cross_entropy_error(y,t):
    delta = 1e-7 # 로그 수식 안 값이 0이 되지 않게 하기 위함
    return -np.sum(t*np.log(y+delta))

In [11]:
t = [0,0,1,0,0,0,0,0,0,0] # 2 원-핫 인코딩

y = [0.1,0.05,0.6,.0,.05,.1,.0,.1,.0,.0]

cross_entropy_error(np.array(y),np.array(t))

0.510825457099338

In [12]:
y = [.1,.05,.1,.0,.05,.1,.0,.6,.0,.0] # 신경망의 출력값이 7이 가장 높은 경우 

cross_entropy_error(np.array(y),np.array(t))


2.302584092994546

### 미니배치(mini-batch)